In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [2]:
# read data from file
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv")

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [3]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [4]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [5]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [6]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [7]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [8]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [9]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [10]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [11]:
train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [12]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

(460, 17)
Index(['PhoneId', 'Screen to Body Ratio (calculated)', 'Processor_frequency',
       'Internal Memory', 'Num_cores', 'RAM', 'Sim1', 'os_name', 'Resolution',
       'Capacity', 'SIM Slot(s)', 'Weight', 'Pixel Density', 'Brand', 'Height',
       'SIM 2', 'Screen Size'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Screen to Body Ratio (calculated)', 'Processor_frequency',
       'Internal Memory', 'RAM', 'Resolution', 'Capacity', 'Weight',
       'Pixel Density', 'Height', 'Screen Size', 'Num_cores_312',
       'Num_cores_Deca', 'Num_cores_Dual', 'Num_cores_Hexa', 'Num_cores_Octa',
       'Num_cores_Other', 'Num_cores_Quad', 'Num_cores_Tru-Octa', 'Sim1_2G',
       'Sim1_3G', 'Sim1_4G', 'os_name_Android', 'os_name_Blackberry',
       'os_name_KAI', 'os_name_Nokia', 'os_name_Other', 'os_name_Tizen',
       'os_name_iOS', 'SIM Slot(s)_Dual SIM, GSM+CDMA',
       'SIM Slot(s)_Dual SIM, GSM+GSM',
       'SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE',
       'SIM Slot(s)_Single SI

In [13]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [106]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [107]:
train_new.head()

,PhoneId,Screen to Body Ratio (calculated),Processor_frequency,Internal Memory,RAM,Resolution,Capacity,Weight,Pixel Density,Height,Screen Size,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,Sim1_2G,Sim1_3G,Sim1_4G,os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,Brand_Jivi,Brand_Karbonn,Brand_LG,Brand_Lava,Brand_LeEco,Brand_Lenovo,Brand_Lephone,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,target
0,0,80.68,1.8,64,4,20,4000,182,403,157.9,6.26,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1
1,1,80.85,1.8,32,3,8,4230,168,271,156.2,6.20,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,2,83.68,2.1,32,3,25,3500,168,409,157.0,6.30,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,4,74.78,2.2,64,4,24,3300,169,411,159.8,6.00,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,5,84.23,2.2,64,4,16,3750,175,396,160.4,6.50,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [137]:
test_new.to_csv("te.csv")

In [108]:
test_new.head()
tn=test_new.drop(['PhoneId'],axis=1)
#tn=tn.drop(['Num_cores_Other','Num_cores_312','os_name_Blackberry','Brand_LeEco','os_name_KAI','Brand_Spice','Brand_Reliance','Brand_Billion','Brand_Lephone','Brand_Do'],axis=1)

In [109]:
data=train_new


In [110]:

x=data.Rating>=THRESHOLD
data['target']=x*1
data.drop('Rating',axis=1,inplace=True)

AttributeError: 'DataFrame' object has no attribute 'Rating'

In [111]:
print(data['target'].value_counts())


1    238
0    103
Name: target, dtype: int64


In [129]:
import pandas as pd
from IPython.display import display
#df=data.groupby('target').mean()
#pd.options.display.max_columns = None
#display(df)
print(data.groupby(['target'], sort=False)['Internal Memory'].max())
print(data.groupby(['target'], sort=False)['Screen to Body Ratio (calculated)'].max())
print(data.groupby(['target'], sort=False)['Capacity'].max())
print(data.groupby(['target'], sort=False)['RAM'].max())
print(data.groupby(['target'], sort=False)['Height'].max())
print(data.groupby(['target'], sort=False)['Weight'].max())
print(data.groupby(['target'], sort=False)['Pixel Density'].max())
print(data.groupby(['target'], sort=False)['Screen Size'].max())
print(data.groupby(['target'], sort=False)['Resolution'].max())
print(data.groupby(['target'], sort=False)['Processor_frequency'].max())

target
1    512
0    128
Name: Internal Memory, dtype: int64
target
1    87.66
0    80.80
Name: Screen to Body Ratio (calculated), dtype: float64
target
1    13000
0     5000
Name: Capacity, dtype: int64
target
1     64
0    512
Name: RAM, dtype: int64
target
1    174.1
0    164.2
Name: Height, dtype: float64
target
1    330
0    207
Name: Weight, dtype: int64
target
1    576
0    554
Name: Pixel Density, dtype: int64
target
1    6.59
0    6.26
Name: Screen Size, dtype: float64
target
1    25
0    20
Name: Resolution, dtype: int64
target
1    2.80
0    2.45
Name: Processor_frequency, dtype: float64


In [113]:
X=data.drop(['PhoneId','target'],axis=1)
Y=train_new.target


In [114]:
X['RAM'] = X['RAM'].apply(lambda x: x/1024 if x>32 else x)
tn['RAM'] = tn['RAM'].apply(lambda x: x/1024 if x>32 else x)


In [115]:
X['Capacity'] = X['Capacity'].apply(lambda x:0 if x>5000 else 1)
X['Screen to Body Ratio (calculated)'] = X['Screen to Body Ratio (calculated)'].apply(lambda x:1 if x>80.8 else 0)
X['RAM'] = X['RAM'].apply(lambda x:1 if x>0.5 else 0)
X['Internal Memory'] = X['Internal Memory'].apply(lambda x:1 if x>128 else 0)
X['Processor_frequency'] = X['Processor_frequency'].apply(lambda x:1 if x>2.45 else 0)
X['Height'] = X['Height'].apply(lambda x:1 if x>164.2 else 0)
X['Weight'] = X['Weight'].apply(lambda x:1 if x>207 else 0)
X['Pixel Density'] = X['Pixel Density'].apply(lambda x:1 if x>554 else 0)
X['Screen Size'] = X['Screen Size'].apply(lambda x:1 if x>6.26 else 0)
X['Resolution'] = X['Resolution'].apply(lambda x:1 if x>20 else 0)



In [819]:
'''X['Capacity'] = X['Capacity'].apply(lambda x:1 if x>4000 else 0)
X['Screen to Body Ratio (calculated)'] = X['Screen to Body Ratio (calculated)'].apply(lambda x:1 if x>80 else 0)
X['RAM'] = X['RAM'].apply(lambda x:1 if x>=2 else 0)
X['Internal Memory'] = X['Internal Memory'].apply(lambda x:1 if x>=40 else 0)
X['Processor_frequency'] = X['Processor_frequency'].apply(lambda x:1 if x>2 else 0)
X['Height'] = X['Height'].apply(lambda x:1 if x>160 else 0)
X['Weight'] = X['Weight'].apply(lambda x:1 if x>158 else 0)
X['Pixel Density'] = X['Pixel Density'].apply(lambda x:1 if x>=365 else 0)
X['Screen Size'] = X['Screen Size'].apply(lambda x:1 if x>=5.5 else 0)
X['Resolution'] = X['Resolution'].apply(lambda x:1 if x>15 else 0)


'''

"X['Capacity'] = X['Capacity'].apply(lambda x:1 if x>4000 else 0)\nX['Screen to Body Ratio (calculated)'] = X['Screen to Body Ratio (calculated)'].apply(lambda x:1 if x>80 else 0)\nX['RAM'] = X['RAM'].apply(lambda x:1 if x>=2 else 0)\nX['Internal Memory'] = X['Internal Memory'].apply(lambda x:1 if x>=40 else 0)\nX['Processor_frequency'] = X['Processor_frequency'].apply(lambda x:1 if x>2 else 0)\nX['Height'] = X['Height'].apply(lambda x:1 if x>160 else 0)\nX['Weight'] = X['Weight'].apply(lambda x:1 if x>158 else 0)\nX['Pixel Density'] = X['Pixel Density'].apply(lambda x:1 if x>=365 else 0)\nX['Screen Size'] = X['Screen Size'].apply(lambda x:1 if x>=5.5 else 0)\nX['Resolution'] = X['Resolution'].apply(lambda x:1 if x>15 else 0)\n\n\n"

In [116]:
X=X.drop(['Num_cores_312','os_name_Blackberry','Brand_LeEco','os_name_KAI','os_name_Nokia','os_name_Tizen','Brand_Spice','Brand_Reliance','Brand_Billion','Brand_Lephone','Brand_Do','SIM Slot(s)_Dual SIM, GSM+CDMA'],axis=1)
tn=tn.drop(['Num_cores_312','os_name_Blackberry','Brand_LeEco','os_name_KAI','os_name_Nokia','os_name_Tizen','Brand_Spice','Brand_Reliance','Brand_Billion','Brand_Lephone','Brand_Do','SIM Slot(s)_Dual SIM, GSM+CDMA'],axis=1)

#X=X.drop(['Num_cores_312','Brand_Jivi','os_name_Other','SIM Slot(s)_Dual SIM, GSM+CDMA','Brand_LeEco','Brand_Huawei','Brand_OnePlus','os_name_Nokia','Num_cores_Tru-Octa','Brand_Spice','Brand_Billion','Brand_Realme','Brand_Xiaomi Poco','Brand_Lephone','Brand_Do','Brand_Google','Brand_Lyf'],axis=1)
#tn=tn.drop(['Num_cores_312','Brand_Jivi','os_name_Other','SIM Slot(s)_Dual SIM, GSM+CDMA','Brand_LeEco','Brand_Huawei','Brand_OnePlus','os_name_Nokia','Num_cores_Tru-Octa','Brand_Spice','Brand_Billion','Brand_Realme','Brand_Xiaomi Poco','Brand_Lephone','Brand_Do','Brand_Google','Brand_Lyf'],axis=1)

#X=X[['Height','Processor_frequency','Weight','Screen to Body Ratio (calculated)','Capacity','Pixel Density','Screen Size','Internal Memory','RAM','Resolution','Brand_Micromax','Brand_Samsung','SIM 2_4G','SIM Slot(s)_Dual SIM, GSM+GSM','Brand_Vivo','SIM 2_2G','Brand_HTC','Num_cores_Quad','Brand_Moto']]
#tn=tn[['Height','Processor_frequency','Weight','Screen to Body Ratio (calculated)','Capacity','Pixel Density','Screen Size','Internal Memory','RAM','Resolution','Brand_Micromax','Brand_Samsung','SIM 2_4G','SIM Slot(s)_Dual SIM, GSM+GSM','Brand_Vivo','SIM 2_2G','Brand_HTC','Num_cores_Quad','Brand_Moto']]


In [117]:
X=X.values
Y= Y.values

In [118]:
class MPNeuron:
  
  def __init__(self):
    self.b = None
    
  def model(self, x):
    return(sum(x) >= self.b)
  
  def predict(self, X):
    Y = []
    for x in X:
      result = self.model(x)
      Y.append(result)
    return np.array(Y)
  
  def fit(self, X, Y):
    accuracy = {}
    
    for b in range(X.shape[1] + 1):
      self.b = b
      Y_pred = self.predict(X)
      accuracy[b] = accuracy_score(Y_pred, Y)
      
    best_b = max(accuracy, key = accuracy.get)
    self.b = best_b
    
    print('Optimal value of b is', best_b)
    print('Highest accuracy is', accuracy[best_b])

In [119]:
mp_neuron = MPNeuron()
mp_neuron.fit(X,Y)

Optimal value of b is 7
Highest accuracy is 0.7126099706744868


In [120]:
## Import the random forest model.
from sklearn.ensemble import RandomForestClassifier 
## This line instantiates the model. 
rf = RandomForestClassifier() 
## Fit the model on your training data.
rf.fit(X,Y) 
## And score it on your testing data.
rf.score(X,Y)

0.9002932551319648

In [121]:
import pandas as pd
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index= X.columns,
                                    columns=['importance']).sort_values('importance',  ascending=False)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [122]:
li=[]
feature_importances.x=(feature_importances.importance>0.006)*1
import pandas as pd
from IPython.display import display
pd.options.display.max_rows = None
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(X.columns, feature_importances.importance):
    feats[feature] = importance #add the name/value pair 
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
print(importances.sort_values(by='Gini-importance'))


NameError: name 'feature_importances' is not defined

In [130]:
tn1=tn.values

In [131]:
Y_test_predz = mp_neuron.predict(tn1)
Y_test_predz=Y_test_predz*1


In [132]:
Target=pd.DataFrame(Y_test_predz)
tn['Target'] = Target.values


In [135]:
print(tn['Target'].value_counts())


1    119
Name: Target, dtype: int64


In [134]:
test_new.shape[0]

119

## Dummy Solution

In [120]:
submission = pd.DataFrame({'PhoneId':test_new['PhoneId'], 'Class':tn['Target']})
submission = submission[['PhoneId', 'Class']]
submission.head()

,PhoneId,Class
0,3,1
1,11,1
2,13,1
3,16,1
4,19,1


In [121]:
submission.to_csv("submission.csv", index=False)

In [38]:
"""
WRITE YOUR MODELLING CODE HERE
"""

'\nWRITE YOUR MODELLING CODE HERE\n'